In [1]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup

## Võtame iga erakonna näitelingist perioodide raportite id-d

In [25]:
def get_soup(link):
    """returns beautifulsoup object from link content
    -INPUT:
        -link: str of url to be requested
    -OUTPUT:
        - beautifulsoup of link content"""
    soup=BeautifulSoup(requests.get(link).content)
    return soup

In [39]:
def get_party_time_ids(link):
    soup=get_soup(link)
    options=soup.find_all('select')[1].find_all('option')
    time_ids=[]
    for option in options:
        time_id=option.get('value')
        if time_id!='':
            time_ids.append(option.get('value'))
    return time_ids

In [45]:
#iga erakonna näidislink, siit saan iga erakonna iga raporti perioodi id
#siia on vaja lisada uusi näiteid, kui tekib mõni uus erakond
start_links=['https://erjk.ee/et/aruanded/erakondade-tulud-ja-laekumised?report=2400',
            'https://erjk.ee/et/aruanded/erakondade-tulud-ja-laekumised?report=421173',
            'https://erjk.ee/et/aruanded/erakondade-tulud-ja-laekumised?report=421803',
            'https://erjk.ee/et/aruanded/erakondade-tulud-ja-laekumised?report=422296',
            'https://erjk.ee/et/aruanded/erakondade-tulud-ja-laekumised?report=420100',
            'https://erjk.ee/et/aruanded/erakondade-tulud-ja-laekumised?report=371680',
            'https://erjk.ee/et/aruanded/erakondade-tulud-ja-laekumised?report=371677',
            'https://erjk.ee/et/aruanded/erakondade-tulud-ja-laekumised?report=2550',
            'https://erjk.ee/et/aruanded/erakondade-tulud-ja-laekumised?report=371667',
            'https://erjk.ee/et/aruanded/erakondade-tulud-ja-laekumised?report=423404',
            'https://erjk.ee/et/aruanded/erakondade-tulud-ja-laekumised?report=422808',
            'https://erjk.ee/et/aruanded/erakondade-tulud-ja-laekumised?report=2555',
            'https://erjk.ee/et/aruanded/erakondade-tulud-ja-laekumised?report=422893',
            'https://erjk.ee/et/aruanded/erakondade-tulud-ja-laekumised?report=2556',
            'https://erjk.ee/et/aruanded/erakondade-tulud-ja-laekumised?report=423067']

period_ids=[]
for start_link in start_links:
    period_id=get_party_time_ids(start_link)
    period_ids.extend(period_id)

In [48]:
len(period_ids), len(set(period_ids))

(393, 393)

In [49]:
period_ids[:5]

['2400', '2399', '2398', '2205', '2101']

## Kõikide perioodide id-de kohta tõmbame raportid alla

In [2]:
def get_query_jsn(url):
    """returns json from request
    -INPUT:
        -url: str of url to be requested
    -OUTPUT:
        - json formatted data"""
    return json.loads(requests.get(url).content)

In [23]:
def get_page_data(jsn):
    party=jsn[0]['title']
    groups=jsn[0]['groups'][0]
    df=pd.DataFrame(groups['data'])
    df.columns=['laekumise_aeg', 'tululiik', 'nimi', 'sünnikuupäev_regkood', 'summa']
    df['erakond']=party
    return df

In [53]:
df_reports=pd.DataFrame()
for i, period_id in enumerate(period_ids):
    if i%10==0:
        print(f'working on report nr {i}')
    url=f'https://erjk.ee/et/api/erjk/quarterly_revenues/{period_id}'
    try:
        jsn=get_query_jsn(url)
        df_period=get_page_data(jsn)
        df_reports=pd.concat([df_reports, df_period])
    except Exception as e:
        print(e)
        print(url)

working on report nr 0
Length mismatch: Expected axis has 0 elements, new values have 5 elements
https://erjk.ee/et/api/erjk/quarterly_revenues/2398
Length mismatch: Expected axis has 0 elements, new values have 5 elements
https://erjk.ee/et/api/erjk/quarterly_revenues/2205
Length mismatch: Expected axis has 0 elements, new values have 5 elements
https://erjk.ee/et/api/erjk/quarterly_revenues/2073
Length mismatch: Expected axis has 0 elements, new values have 5 elements
https://erjk.ee/et/api/erjk/quarterly_revenues/2068
Length mismatch: Expected axis has 0 elements, new values have 5 elements
https://erjk.ee/et/api/erjk/quarterly_revenues/2057
working on report nr 10
Length mismatch: Expected axis has 0 elements, new values have 5 elements
https://erjk.ee/et/api/erjk/quarterly_revenues/2013
working on report nr 20
working on report nr 30
working on report nr 40
working on report nr 50
working on report nr 60
working on report nr 70
working on report nr 80
working on report nr 90
worki

In [54]:
df_reports.shape

(127386, 6)

In [60]:
df_reports=df_reports.dropna()
df_reports.shape

(127386, 6)

In [78]:
#eraldame pratei nime ja aruande perioodi
df_reports=pd.concat([df_reports, df_reports['erakond'].apply(lambda x: pd.Series(str(x).split(' ', 1)))],  axis='columns')

In [80]:
df_reports.columns=[ 'laekumise_aeg', 'tululiik', 'nimi', 'sünnikuupäev_regkood',
                    'summa', 'periood_erakond', 'aruande_periood', 'erakond']

In [81]:
df_reports.head()

,laekumise_aeg,tululiik,nimi,sünnikuupäev_regkood,summa,periood_erakond,aruande_periood,erakond
0,25.06.2018,Liikmemaks,SVEN SILDNIK,03.01.1964,20,26.09.2018 Eesti Iseseisvuspartei,26.09.2018,Eesti Iseseisvuspartei
0,21.03.2018,Liikmemaks,SVEN SILDNIK,03.01.1964,20,26.09.2018 Eesti Iseseisvuspartei,26.09.2018,Eesti Iseseisvuspartei
0,06.06.2017,Liikmemaks,TÕNIS TULP,20.07.1951,25,10.07.2017 Eesti Iseseisvuspartei,10.07.2017,Eesti Iseseisvuspartei
0,09.01.2017,Rahaline annetus,KARL MARTINSON,30.06.1979,30,13.04.2017 Eesti Iseseisvuspartei,13.04.2017,Eesti Iseseisvuspartei
0,24.03.2016,Rahaline annetus,KARL MARTINSON,30.06.1979,20,11.04.2016 Eesti Iseseisvuspartei,11.04.2016,Eesti Iseseisvuspartei


In [82]:
df_reports.tail()

,laekumise_aeg,tululiik,nimi,sünnikuupäev_regkood,summa,periood_erakond,aruande_periood,erakond
615,01.01.2013,Liikmemaks,AADO LINTROP,09.06.1956,7,18.06.2013 Sotsiaaldemokraatlik Erakond,18.06.2013,Sotsiaaldemokraatlik Erakond
616,01.01.2013,Liikmemaks,AHTI NOOR,18.09.1985,3,18.06.2013 Sotsiaaldemokraatlik Erakond,18.06.2013,Sotsiaaldemokraatlik Erakond
617,01.01.2013,Liikmemaks,JAAN SÕRRA,30.10.1954,20,18.06.2013 Sotsiaaldemokraatlik Erakond,18.06.2013,Sotsiaaldemokraatlik Erakond
618,01.01.2013,Liikmemaks,MARGUS TOOMLA,21.10.1958,5,18.06.2013 Sotsiaaldemokraatlik Erakond,18.06.2013,Sotsiaaldemokraatlik Erakond
619,01.01.2013,Liikmemaks,TANEL KÄÄMBRE,26.11.1970,10,18.06.2013 Sotsiaaldemokraatlik Erakond,18.06.2013,Sotsiaaldemokraatlik Erakond


In [83]:
df_reports.describe(include='all')

,laekumise_aeg,tululiik,nimi,sünnikuupäev_regkood,summa,periood_erakond,aruande_periood,erakond
count,127386,127386,127386,127386,127386.000000,127386,127386,127386
unique,3462,5,25510,15634,NaN,348,182,15
top,21.06.2016,Liikmemaks,Eesti Konservatiivne Rahvaerakond,80040344,NaN,10.04.2018 Eesti Keskerakond,10.04.2018,Eesti Keskerakond
freq,731,86274,1067,1067,NaN,2328,5237,38414
mean,NaN,NaN,NaN,NaN,565.042642,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,9207.862250,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,-50000.000000,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,5.000000,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,12.000000,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,25.000000,NaN,NaN,NaN


In [84]:
df_reports[df_reports.summa<0]

,laekumise_aeg,tululiik,nimi,sünnikuupäev_regkood,summa,periood_erakond,aruande_periood,erakond
9,22.06.2022,Liikmemaks,EVA-MARIA LIIMETS,31.05.1974,-687,08.07.2022 Eesti Keskerakond,08.07.2022,Eesti Keskerakond
15,17.06.2022,Liikmemaks,KRISTIAN JAANI,11.12.1976,-767,08.07.2022 Eesti Keskerakond,08.07.2022,Eesti Keskerakond
372,07.09.2021,Rahaline annetus,MARK JEFIMOV,13.07.2002,-300,03.06.2022 Eesti Keskerakond,03.06.2022,Eesti Keskerakond
647,20.07.2021,Rahaline annetus,VLADIMIR ARHIPOV,26.09.1955,-5000,03.06.2022 Eesti Keskerakond,03.06.2022,Eesti Keskerakond
131,08.06.2020,Rahaline annetus,JANA-HELEN JUHASTE,20.04.1985,-50000,10.07.2020 Eesti Keskerakond,10.07.2020,Eesti Keskerakond


## Salvesta

In [85]:
df_reports.to_csv('data//erakondaded_rahastamine.csv', index=False)